In [1]:
import os
os.chdir('D:\engsci\year 3\CLASS\APS360\Project') #Sets current working directory! #Set working directory!
import json

# This file contains the required code to compile all the files from the dataset, convert them into arrays and save this new dataset.

Will return data as: (number of songs,number of samples,number of features)

In [2]:
# import MIDI_backconversion #Would be cleaner. Oh well.
# import MIDI_conversion2

import mido
from mido import MidiFile, Message, MidiTrack
import torch
import torchvision
import os
import numpy as np
import time
import midi
import glob  
from tqdm import tqdm
import numpy as np
# import torchaudio

# Defining required functions

In [3]:
# def Numpy2Midi(array):                      # Defining it iteratively...
#     #Start with initializing a new Mido Track:
#     mid = MidiFile()
#     track = MidiTrack()
#     already_added = []
#     for i,note in enumerate(array):         # Get the index and the note. Array must be int array             
#         #print(note)
#         sustain = note[3]                   # Get how long the note is on for.
#         #print(sustain)
#         j = 1
#         track.append(Message('note_on',note = array[i,0], velocity = array[i,1],time = array[i,2])) # Add the note to the track.
#         while (i+j <= array.shape[0]-1) and (sustain > array[i+j,2]):       
#             # ^^ If sustain is longer than the time to the next note or we reached the last note...
#             if (i+j) not in already_added:
# #                 print('not already added')
#                 track.append(Message('note_on',note = array[i+j,0], velocity = array[i+j,1],time = array[i+j,2]))
#                 already_added.append([i+j])
# #                 print('updated already_apped list')
#             sustain = sustain - array[i+j,2] # Subtract the time to the next note.
#             j += 1 #update index to check agains.
#         track.append(Message('note_on',note = array[i,0], velocity = 0,time = sustain)) # Add the off note.
#     mid.tracks.append(track)
#     return mid

def Midi2Numpy(mid):                                #converts to numpy array removing non-note messages
    track = mid.tracks[1]                           #0th track only contains meta-messages, all notes on 1st track
    notes = np.empty([0,4])                         #4 because we're tracking: note, velocity,time and sustain.
    time = 0
    for msg in track:
        if msg.type == "note_on":                   # only count "note" messages - other inputs i.e. foot pedals are ignored
            #if msg[1] > 0: ?
            notes = np.append(notes,np.array([[msg.note, msg.velocity, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0
        else:
            time += msg.time                        #adjust time when removing other messages (time is length of note? start time of notes from start of song?)
    return notes


def NumpyGetSustain(notes):                       # Removes the zeroes
    for i, msg in enumerate(notes):
        if msg[1] > 0:                            # if velocity is not 0 (if note is being turned on)
            j = 1
            sustain = 0
            while msg[0] != notes[i+j][0]:        # while note values are different
                sustain += notes[i+j][2]          # Update sustain of ith note with i+jth note's time.
                j += 1                            #search for next message with same note i.e. message telling that note was released
            notes[i,3] = sustain + notes[i+j][2]  # once next appearance of note identified, updat ith note's sustain as time on until next note.
    time = 0
    for i, msg in enumerate(notes):               
        if msg[1] > 0:                            # If velocity is not 0
            notes[i,2] += time                    # Update the ith note's time
            time = 0
        else:
            time += msg[2]                        #adjust time
    notes = notes[notes[:,1] > 0]                 #filter for notes with positive velocities (note presses)
    return notes


def CountTracks(directory):          #Count files and tracks in folder
    trackCount = 0
    fileCount = 0
    for file in os.listdir(directory):
        if file.endswith(".midi"):
            fileCount += 1
            midiDir = MidiFile(directory+"/"+file)
            for track in midiDir.tracks: # Locates the tracks in a 'song.midi'
                trackCount += 1 # Counts tracks.
    print(fileCount+" files")
    print(trackCount+" tracks")

    
def PrintMessages(mid):                # print midi messages
    for i, track in enumerate(mid.tracks): # Iterates through tracks and also outputs the index.
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            print(msg)

            
def PrintSomeMessages(mid):             #print first 200 midi messages
    track = mid.tracks[1]
    for i,msg in enumerate(track):
        if i < 200:
            print(msg)
            
def cleanupMessages(mid):
    track = mid.tracks[1]
    track2 = []
    for msg in track:
        if msg.type == "note_on":
            track2.append(msg)
    mid.tracks[1] = track2
            

In [4]:
#Note: The dataset needs to be in the same directory. Set working directory above.
# Process: 
# 1) Import the dataset (likely requires a dataloader so it can be done in batches)
# - Will require unzipping it into the machine
# - Will then require loading it into a variable.
# 2) For each entry of the dataset, call MIDI2Numpy on the datasample.
# 3) Save the outputs (Again, likely need to do this in batches if dataset too large)

In [5]:
#Only if required (using colab)
# !unzip '\datasets\maestro-v2.0.0-midi' -d '/root/datasets' #Customize data directory to your machine.

In [6]:
#Will use DatasetFolder: https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder (scroll up or ctrl F)
def loader_fnct(input):
    #print('entered loader_fnct')
    song = MidiFile(input) #Using MIDO
    #print(type(song))
    song_array = Midi2Numpy(song)
    song_array = NumpyGetSustain(song_array)
#     print(song_array)
    return song_array



In [7]:
# # The data is split in different year folders. 
# unprocessed_data_file = torchvision.datasets.DatasetFolder('D:/engsci/year 3/CLASS/APS360/Project/datasets/maestro-v2.0.0-midi/maestro-v2.0.0',loader = loader_fnct,extensions = 'midi')
# print('number of songs found: ',len(unprocessed_midis))
# print(unprocessed_midis)
# print('Confirming type: ',type(unprocessed_midis))


In [ ]:
# https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
#Grab all the midi file names and open them one by one...
number_of_notes = 10 #Must be smaller than the smallest midi file...
arr = np.zeros((1,number_of_notes,4)) #Only keep n notes for now... Need to figure out how to deal with varied length data...

with open('D:\engsci\year 3\CLASS\APS360\Project\datasets\maestro-v2.0.0-midi\maestro-v2.0.0\maestro-v2.0.0.json') as json_file:
    data = json.load(json_file)
    for i in range(0,len(data)) :
        print('concatenating... : ',i)
        #print(data[i]['midi_filename'])
        file_name = data[i]['midi_filename']
        #print(file_name)
        song_array = loader_fnct('D:/engsci/year 3/CLASS/APS360/Project/datasets/maestro-v2.0.0-midi/maestro-v2.0.0/{}'.format(file_name))
        print(song_array.shape)
#         print(type(song_array))
        for j in range(0,len(song_array)//10): #Take multiples of number_of_notes
#             temp = np.zeros((1,number_of_notes,4))
#             print(j)
#             print(number_of_notes*(j+1))
#             print(song_array[number_of_notes*j:number_of_notes(,:])
            temp[0] = song_array[number_of_notes*j:number_of_notes*(j+1),:]
            arr = np.concatenate((arr,temp),0)
        #print(arr[i])



concatenating... :  0
(4197, 4)
concatenating... :  1
(4206, 4)
concatenating... :  2
(3326, 4)
concatenating... :  3
(6316, 4)
concatenating... :  4
(4019, 4)
concatenating... :  5
(2354, 4)
concatenating... :  6
(721, 4)
concatenating... :  7
(5362, 4)
concatenating... :  8
(783, 4)
concatenating... :  9
(2828, 4)
concatenating... :  10
(2193, 4)
concatenating... :  11
(5392, 4)
concatenating... :  12
(6498, 4)
concatenating... :  13
(6118, 4)
concatenating... :  14
(6321, 4)
concatenating... :  15
(1052, 4)
concatenating... :  16
(1904, 4)
concatenating... :  17
(6232, 4)
concatenating... :  18
(6196, 4)
concatenating... :  19
(6220, 4)
concatenating... :  20
(6178, 4)
concatenating... :  21
(9486, 4)
concatenating... :  22
(9525, 4)
concatenating... :  23
(9727, 4)
concatenating... :  24
(5600, 4)
concatenating... :  25
(7516, 4)
concatenating... :  26
(7629, 4)
concatenating... :  27
(7910, 4)
concatenating... :  28
(7548, 4)
concatenating... :  29
(7891, 4)
concatenating... :  30

In [ ]:
number_of_notes = 10
temp = np.ones((1,number_of_notes,4))
arr = np.zeros((1,number_of_notes,4))
arr = np.concatenate((arr,temp),0)
print(arr.shape)

In [ ]:
print(arr[1000])

In [ ]:
# unprocessed_data = torch.utils.data.DataLoader(unprocessed_data_file, batch_size=1,num_workers=1,shuffle=False)
# print(len(unprocessed_data))
# data = np.empty((1,1,1),dtype = float)
# for midi, label in iter(unprocessed_data): #Loop through the 1282 songs...
#     break

In [ ]:
#Save the data file!!!
np.save('nparray_dataset.npy',arr)